In [5]:
!nvidia-smi

Mon Jun  2 16:23:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [7]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-87et7g2g
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-87et7g2g
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-4g4xyiuc/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [8]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpigdy3ekv".


In [9]:
%%writefile A_star_seq.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>
#include <float.h>
#include <string.h>
#include <time.h>

#define ROW 1000
#define COL 1000

typedef struct {
    int first;
    int second;
} Pair;

typedef struct {
    double first;
    Pair second;
} pPair;

typedef struct {
    int parent_i, parent_j;
    double f, g, h;
} cell;

bool isValid(int row, int col) {
    return (row >= 0) && (row < ROW) && (col >= 0) && (col < COL);
}

bool isUnBlocked(int** grid, int row, int col) {
    return (grid[row][col] == 1);
}

bool isDestination(int row, int col, Pair dest) {
    return (row == dest.first && col == dest.second);
}

double calculateHValue(int row, int col, Pair dest) {
    return sqrt((row - dest.first) * (row - dest.first) +
                (col - dest.second) * (col - dest.second));
}
void tracePath(cell** cellDetails, Pair dest) {
    printf("\nThe Path is ");
    int row = dest.first;
    int col = dest.second;

    Pair* Path = (Pair*)malloc(ROW * COL * sizeof(Pair));
    int pathLength = 0;
    double totalCost = 0.0;

    while (!(cellDetails[row][col].parent_i == row &&
             cellDetails[row][col].parent_j == col)) {
        Path[pathLength++] = (Pair){row, col};

        int parent_row = cellDetails[row][col].parent_i;
        int parent_col = cellDetails[row][col].parent_j;

        // Compute cost based on movement type
        int dr = abs(row - parent_row);
        int dc = abs(col - parent_col);

        if (dr == 1 && dc == 1)
            totalCost += 1.414; // Diagonal movement
        else
            totalCost += 1.0;   // Straight movement

        row = parent_row;
        col = parent_col;
    }

    Path[pathLength++] = (Pair){row, col}; // Add the starting point

    for (int i = pathLength - 1; i >= 0; i--) {
        printf("-> (%d,%d) ", Path[i].first, Path[i].second);
    }

    printf("\nTotal path cost: %.3f\n", totalCost);

    free(Path);
}


typedef struct {
    pPair* elements;
    int size;
    int capacity;
} OpenList;

void initOpenList(OpenList* list, int capacity) {
    list->elements = (pPair*)malloc(capacity * sizeof(pPair));
    list->size = 0;
    list->capacity = capacity;
}

void insertOpenList(OpenList* list, pPair element) {
    if (list->size == list->capacity) {
        list->capacity *= 2;
        list->elements = (pPair*)realloc(list->elements, list->capacity * sizeof(pPair));
    }
    list->elements[list->size++] = element;
}

pPair extractMin(OpenList* list) {
    int minIndex = 0;
    for (int i = 1; i < list->size; i++) {
        if (list->elements[i].first < list->elements[minIndex].first) {
            minIndex = i;
        }
    }
    pPair minElement = list->elements[minIndex];
    list->elements[minIndex] = list->elements[--list->size];
    return minElement;
}

bool isEmpty(OpenList* list) {
    return list->size == 0;
}

void freeOpenList(OpenList* list) {
    free(list->elements);
}

void aStarSearch(int** grid, Pair src, Pair dest) {
    clock_t start_time = clock();

    if (!isValid(src.first, src.second) || !isValid(dest.first, dest.second)) {
        printf("Source or Destination is invalid\n");
        return;
    }

    if (!isUnBlocked(grid, src.first, src.second) || !isUnBlocked(grid, dest.first, dest.second)) {
        printf("Source or Destination is blocked\n");
        return;
    }

    if (isDestination(src.first, src.second, dest)) {
        printf("Already at the destination\n");
        return;
    }

    bool** closedList = (bool**)malloc(ROW * sizeof(bool*));
    for (int i = 0; i < ROW; i++) {
        closedList[i] = (bool*)calloc(COL, sizeof(bool));
    }

    cell** cellDetails = (cell**)malloc(ROW * sizeof(cell*));
    for (int i = 0; i < ROW; i++) {
        cellDetails[i] = (cell*)malloc(COL * sizeof(cell));
        for (int j = 0; j < COL; j++) {
            cellDetails[i][j].f = FLT_MAX;
            cellDetails[i][j].g = FLT_MAX;
            cellDetails[i][j].h = FLT_MAX;
            cellDetails[i][j].parent_i = -1;
            cellDetails[i][j].parent_j = -1;
        }
    }

    int i = src.first, j = src.second;
    cellDetails[i][j].f = cellDetails[i][j].g = cellDetails[i][j].h = 0.0;
    cellDetails[i][j].parent_i = i;
    cellDetails[i][j].parent_j = j;

    OpenList openList;
    initOpenList(&openList, 10000);
    insertOpenList(&openList, (pPair){0.0, {i, j}});

    bool foundDest = false;
    int directions[8][2] = {{-1, 0}, {1, 0}, {0, 1}, {0, -1},
                            {-1, 1}, {-1, -1}, {1, 1}, {1, -1}};
    double costs[8] = {1.0, 1.0, 1.0, 1.0, 1.414, 1.414, 1.414, 1.414};

    while (!isEmpty(&openList)) {
        pPair p = extractMin(&openList);
        i = p.second.first;
        j = p.second.second;
        closedList[i][j] = true;

        for (int k = 0; k < 8; k++) {
            int new_i = i + directions[k][0];
            int new_j = j + directions[k][1];

            if (isValid(new_i, new_j)) {
                if (isDestination(new_i, new_j, dest)) {
                    cellDetails[new_i][new_j].parent_i = i;
                    cellDetails[new_i][new_j].parent_j = j;
                    printf("Destination found!\n");
                    tracePath(cellDetails, dest);
                    foundDest = true;

                    clock_t end_time = clock();
                    double time_taken = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
                    printf("\n\nExecution time: %f seconds\n", time_taken);

                    goto cleanup;
                }

                if (!closedList[new_i][new_j] && isUnBlocked(grid, new_i, new_j)) {
                    double gNew = cellDetails[i][j].g + costs[k];
                    double hNew = calculateHValue(new_i, new_j, dest);
                    double fNew = gNew + hNew;

                    if (cellDetails[new_i][new_j].f > fNew) {
                        insertOpenList(&openList, (pPair){fNew, {new_i, new_j}});
                        cellDetails[new_i][new_j].f = fNew;
                        cellDetails[new_i][new_j].g = gNew;
                        cellDetails[new_i][new_j].h = hNew;
                        cellDetails[new_i][new_j].parent_i = i;
                        cellDetails[new_i][new_j].parent_j = j;
                    }
                }
            }
        }
    }

    if (!foundDest) {
        printf("Failed to find the destination\n");
        clock_t end_time = clock();
        printf("Execution time: %f seconds\n",
               ((double)(end_time - start_time)) / CLOCKS_PER_SEC);
    }

cleanup:
    freeOpenList(&openList);
    for (int i = 0; i < ROW; i++) {
        free(closedList[i]);
        free(cellDetails[i]);
    }
    free(closedList);
    free(cellDetails);
}

int main() {
    // Allocate and initialize large grid
    int** grid = (int**)malloc(ROW * sizeof(int*));
    for (int i = 0; i < ROW; i++) {
        grid[i] = (int*)malloc(COL * sizeof(int));
        for (int j = 0; j < COL; j++) {
            grid[i][j] = rand() % 5 == 0 ? 0 : 1; // ~20% blocked
        }
    }

    Pair src = {0, 0};
    Pair dest = {ROW - 1, COL - 1};

    aStarSearch(grid, src, dest);

    for (int i = 0; i < ROW; i++) {
        free(grid[i]);
    }
    free(grid);

    return 0;
}


Writing A_star_seq.c


In [10]:
!gcc -o A_star_seq A_star_seq.c -lm

In [26]:
!./A_star_seq

Destination found!

The Path is -> (0,0) -> (1,1) -> (2,2) -> (3,3) -> (4,4) -> (5,5) -> (6,6) -> (7,7) -> (8,8) -> (9,9) -> (10,10) -> (11,11) -> (12,12) -> (13,13) -> (14,14) -> (15,15) -> (16,16) -> (16,17) -> (17,18) -> (18,19) -> (19,20) -> (20,21) -> (21,22) -> (22,23) -> (23,23) -> (24,24) -> (25,25) -> (26,26) -> (27,27) -> (28,28) -> (29,29) -> (30,30) -> (30,31) -> (30,32) -> (31,33) -> (31,34) -> (32,35) -> (32,36) -> (33,37) -> (34,38) -> (35,39) -> (36,40) -> (37,41) -> (38,42) -> (39,43) -> (40,44) -> (41,45) -> (42,46) -> (42,47) -> (42,48) -> (43,49) -> (43,50) -> (44,51) -> (45,52) -> (46,53) -> (47,54) -> (48,55) -> (49,56) -> (49,57) -> (50,58) -> (51,59) -> (52,60) -> (53,61) -> (54,62) -> (55,63) -> (56,64) -> (57,65) -> (58,66) -> (59,67) -> (60,68) -> (61,68) -> (62,69) -> (62,70) -> (63,71) -> (64,72) -> (64,73) -> (65,74) -> (66,75) -> (67,76) -> (68,77) -> (69,78) -> (70,79) -> (71,80) -> (72,81) -> (73,82) -> (74,83) -> (75,84) -> (76,85) -> (76,86) -> (77,87

Pram

In [13]:
%%writefile A_star_pram.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>
#include <float.h>
#include <string.h>
#include <time.h>
#include <pthread.h>

#define ROW 1000
#define COL 1000
#define NUM_THREADS 10

typedef struct {
    int first;
    int second;
} Pair;

typedef struct {
    double first;
    Pair second;
} pPair;

typedef struct {
    int parent_i, parent_j;
    double f, g, h;
} cell;

typedef struct {
    pPair* elements;
    int size;
    int capacity;
    pthread_mutex_t lock;
} OpenList;

typedef struct {
    int** grid;
    Pair src;
    Pair dest;
    bool** closedList;
    cell** cellDetails;
    OpenList* openList;
    bool foundDest;
    pthread_mutex_t foundMutex;
    pthread_mutex_t listMutex;
    pthread_cond_t cond;
    int activeThreads;
} SharedData;

bool isValid(int row, int col) {
    return (row >= 0) && (row < ROW) && (col >= 0) && (col < COL);
}

bool isUnBlocked(int** grid, int row, int col) {
    return (grid[row][col] == 1);
}

bool isDestination(int row, int col, Pair dest) {
    return (row == dest.first && col == dest.second);
}

double calculateHValue(int row, int col, Pair dest) {
    return sqrt((row - dest.first) * (row - dest.first) +
                (col - dest.second) * (col - dest.second));
}

void tracePath(cell** cellDetails, Pair dest) {
    printf("\nThe Path is ");
    int row = dest.first;
    int col = dest.second;

    Pair* Path = (Pair*)malloc(ROW * COL * sizeof(Pair));
    int pathLength = 0;

    while (!(cellDetails[row][col].parent_i == row &&
             cellDetails[row][col].parent_j == col)) {
        Path[pathLength++] = (Pair){row, col};
        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }

    Path[pathLength++] = (Pair){row, col};

    for (int i = pathLength - 1; i >= 0; i--) {
        printf("-> (%d,%d) ", Path[i].first, Path[i].second);
    }
    free(Path);
}

void initOpenList(OpenList* list, int capacity) {
    list->elements = (pPair*)malloc(capacity * sizeof(pPair));
    list->size = 0;
    list->capacity = capacity;
    pthread_mutex_init(&list->lock, NULL);
}

void insertOpenList(OpenList* list, pPair element) {
    pthread_mutex_lock(&list->lock);
    if (list->size == list->capacity) {
        list->capacity *= 2;
        list->elements = (pPair*)realloc(list->elements, list->capacity * sizeof(pPair));
    }
    list->elements[list->size++] = element;
    pthread_mutex_unlock(&list->lock);
}

pPair extractMin(OpenList* list) {
    pthread_mutex_lock(&list->lock);
    if (list->size == 0) {
        pthread_mutex_unlock(&list->lock);
        return (pPair){DBL_MAX, {-1, -1}};
    }

    int minIndex = 0;
    for (int i = 1; i < list->size; i++) {
        if (list->elements[i].first < list->elements[minIndex].first) {
            minIndex = i;
        }
    }
    pPair minElement = list->elements[minIndex];
    list->elements[minIndex] = list->elements[--list->size];
    pthread_mutex_unlock(&list->lock);
    return minElement;
}

bool isEmpty(OpenList* list) {
    pthread_mutex_lock(&list->lock);
    bool empty = (list->size == 0);
    pthread_mutex_unlock(&list->lock);
    return empty;
}

void freeOpenList(OpenList* list) {
    pthread_mutex_destroy(&list->lock);
    free(list->elements);
}

void* threadAStarSearch(void* arg) {
    SharedData* shared = (SharedData*)arg;
    int directions[8][2] = {{-1, 0}, {1, 0}, {0, 1}, {0, -1},
                           {-1, 1}, {-1, -1}, {1, 1}, {1, -1}};
    double costs[8] = {1.0, 1.0, 1.0, 1.0, 1.414, 1.414, 1.414, 1.414};

    while (1) {
        pthread_mutex_lock(&shared->listMutex);

        // Check if destination found or no more work
        if (shared->foundDest || (isEmpty(shared->openList) && shared->activeThreads == 1)) {
            shared->activeThreads--;
            pthread_mutex_unlock(&shared->listMutex);
            break;
        }

        // Wait for work if queue is empty but other threads are active
        while (!shared->foundDest && isEmpty(shared->openList) && shared->activeThreads > 1) {
            shared->activeThreads--;
            pthread_cond_wait(&shared->cond, &shared->listMutex);
            shared->activeThreads++;
        }

        pPair p = extractMin(shared->openList);
        pthread_mutex_unlock(&shared->listMutex);

        if (p.second.first == -1) continue;

        int i = p.second.first;
        int j = p.second.second;

        // Skip if already processed
        pthread_mutex_lock(&shared->listMutex);
        if (shared->closedList[i][j]) {
            pthread_mutex_unlock(&shared->listMutex);
            continue;
        }
        shared->closedList[i][j] = true;
        pthread_mutex_unlock(&shared->listMutex);

        for (int k = 0; k < 8; k++) {
            int new_i = i + directions[k][0];
            int new_j = j + directions[k][1];

            if (isValid(new_i, new_j)) {
                if (isDestination(new_i, new_j, shared->dest)) {
                    pthread_mutex_lock(&shared->foundMutex);
                    if (!shared->foundDest) {
                        shared->foundDest = true;
                        shared->cellDetails[new_i][new_j].parent_i = i;
                        shared->cellDetails[new_i][new_j].parent_j = j;
                        printf("Thread found destination at (%d,%d)\n", new_i, new_j);
                    }
                    pthread_mutex_unlock(&shared->foundMutex);
                    pthread_cond_broadcast(&shared->cond);
                    return NULL;
                }

                pthread_mutex_lock(&shared->listMutex);
                bool isClosed = shared->closedList[new_i][new_j];
                bool isBlocked = !isUnBlocked(shared->grid, new_i, new_j);
                pthread_mutex_unlock(&shared->listMutex);

                if (!isClosed && !isBlocked) {
                    double gNew = shared->cellDetails[i][j].g + costs[k];
                    double hNew = calculateHValue(new_i, new_j, shared->dest);
                    double fNew = gNew + hNew;

                    pthread_mutex_lock(&shared->listMutex);
                    if (shared->cellDetails[new_i][new_j].f > fNew) {
                        shared->cellDetails[new_i][new_j].f = fNew;
                        shared->cellDetails[new_i][new_j].g = gNew;
                        shared->cellDetails[new_i][new_j].h = hNew;
                        shared->cellDetails[new_i][new_j].parent_i = i;
                        shared->cellDetails[new_i][new_j].parent_j = j;
                        insertOpenList(shared->openList, (pPair){fNew, {new_i, new_j}});
                        pthread_cond_signal(&shared->cond);
                    }
                    pthread_mutex_unlock(&shared->listMutex);
                }
            }
        }
    }
    return NULL;
}

void parallelAStarSearch(int** grid, Pair src, Pair dest) {
    clock_t start_time = clock();

    if (!isValid(src.first, src.second) || !isValid(dest.first, dest.second)) {
        printf("Source or Destination is invalid\n");
        return;
    }

    if (!isUnBlocked(grid, src.first, src.second) || !isUnBlocked(grid, dest.first, dest.second)) {
        printf("Source or Destination is blocked\n");
        return;
    }

    if (isDestination(src.first, src.second, dest)) {
        printf("Already at the destination\n");
        return;
    }

    // Initialize shared data
    SharedData shared;
    shared.grid = grid;
    shared.src = src;
    shared.dest = dest;
    shared.foundDest = false;
    shared.activeThreads = NUM_THREADS;
    pthread_mutex_init(&shared.foundMutex, NULL);
    pthread_mutex_init(&shared.listMutex, NULL);
    pthread_cond_init(&shared.cond, NULL);

    // Initialize closed list
    shared.closedList = (bool**)malloc(ROW * sizeof(bool*));
    for (int i = 0; i < ROW; i++) {
        shared.closedList[i] = (bool*)calloc(COL, sizeof(bool));
    }

    // Initialize cell details
    shared.cellDetails = (cell**)malloc(ROW * sizeof(cell*));
    for (int i = 0; i < ROW; i++) {
        shared.cellDetails[i] = (cell*)malloc(COL * sizeof(cell));
        for (int j = 0; j < COL; j++) {
            shared.cellDetails[i][j].f = FLT_MAX;
            shared.cellDetails[i][j].g = FLT_MAX;
            shared.cellDetails[i][j].h = FLT_MAX;
            shared.cellDetails[i][j].parent_i = -1;
            shared.cellDetails[i][j].parent_j = -1;
        }
    }

    int i = src.first, j = src.second;
    shared.cellDetails[i][j].f = 0.0;
    shared.cellDetails[i][j].g = 0.0;
    shared.cellDetails[i][j].h = 0.0;
    shared.cellDetails[i][j].parent_i = i;
    shared.cellDetails[i][j].parent_j = j;

    // Initialize open list
    shared.openList = (OpenList*)malloc(sizeof(OpenList));
    initOpenList(shared.openList, 10000);
    insertOpenList(shared.openList, (pPair){0.0, {i, j}});

    // Create threads
    pthread_t threads[NUM_THREADS];
    for (int t = 0; t < NUM_THREADS; t++) {
        pthread_create(&threads[t], NULL, threadAStarSearch, &shared);
    }

    // Wait for threads to complete
    for (int t = 0; t < NUM_THREADS; t++) {
        pthread_join(threads[t], NULL);
    }

    if (shared.foundDest) {
        printf("\nDestination found!\n");
        tracePath(shared.cellDetails, dest);
    } else {
        printf("\nFailed to find the destination\n");
    }

    clock_t end_time = clock();
    double time_taken = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
    printf("\nExecution time: %f seconds\n", time_taken);

    // Clean up
    freeOpenList(shared.openList);
    free(shared.openList);
    for (int i = 0; i < ROW; i++) {
        free(shared.closedList[i]);
        free(shared.cellDetails[i]);
    }
    free(shared.closedList);
    free(shared.cellDetails);
    pthread_mutex_destroy(&shared.foundMutex);
    pthread_mutex_destroy(&shared.listMutex);
    pthread_cond_destroy(&shared.cond);
}

int main() {
    // Create a more solvable grid (diagonal path with some obstacles)
    int** grid = (int**)malloc(ROW * sizeof(int*));
    for (int i = 0; i < ROW; i++) {
        grid[i] = (int*)malloc(COL * sizeof(int));
        for (int j = 0; j < COL; j++) {
            // Create a mostly open grid with a few obstacles
            if ((i + j) % 7 == 0) {  // ~14% blocked
                grid[i][j] = 0;
            } else {
                grid[i][j] = 1;
            }
        }
    }

    // Ensure start and end are unblocked
    grid[0][0] = 1;
    grid[ROW-1][COL-1] = 1;

    Pair src = {0, 0};
    Pair dest = {ROW - 1, COL - 1};

    printf("Starting A* search from (%d,%d) to (%d,%d)\n",
           src.first, src.second, dest.first, dest.second);
    printf("Grid size: %dx%d\n", ROW, COL);
    printf("Using %d threads\n", NUM_THREADS);

    parallelAStarSearch(grid, src, dest);

    for (int i = 0; i < ROW; i++) {
        free(grid[i]);
    }
    free(grid);

    return 0;
}

Writing A_star_pram.c


In [14]:
!gcc A_star_pram.c -o A_star_pram -lm

In [27]:
!./A_star_pram

Starting A* search from (0,0) to (999,999)
Grid size: 1000x1000
Using 10 threads
Thread found destination at (999,999)

Destination found!

The Path is -> (0,0) -> (1,1) -> (2,2) -> (3,3) -> (4,4) -> (5,5) -> (6,6) -> (7,6) -> (8,7) -> (9,8) -> (10,9) -> (10,10) -> (11,11) -> (12,12) -> (13,13) -> (13,14) -> (14,15) -> (15,16) -> (16,17) -> (17,17) -> (18,18) -> (19,19) -> (20,20) -> (20,21) -> (21,22) -> (22,23) -> (23,24) -> (24,24) -> (25,25) -> (26,26) -> (27,27) -> (27,28) -> (28,29) -> (29,30) -> (30,31) -> (31,31) -> (32,32) -> (33,33) -> (34,34) -> (35,34) -> (36,35) -> (37,36) -> (38,37) -> (39,37) -> (40,38) -> (41,39) -> (42,40) -> (43,40) -> (44,41) -> (45,42) -> (46,43) -> (47,43) -> (48,44) -> (49,45) -> (50,46) -> (50,47) -> (51,48) -> (52,49) -> (53,50) -> (53,51) -> (54,52) -> (55,53) -> (56,54) -> (57,54) -> (58,55) -> (59,56) -> (60,57) -> (60,58) -> (61,59) -> (62,60) -> (63,61) -> (63,62) -> (64,63) -> (65,64) -> (66,65) -> (66,66) -> (67,67) -> (68,68) -> (69,69) 

In [16]:
%%writefile A_star_openmp.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>
#include <float.h>
#include <string.h>
#include <time.h>
#include <omp.h>  // Include OpenMP header

#define ROW 1000
#define COL 1000

typedef struct {
    int first;
    int second;
} Pair;

typedef struct {
    double first;
    Pair second;
} pPair;

typedef struct {
    int parent_i, parent_j;
    double f, g, h;
} cell;

bool isValid(int row, int col) {
    return (row >= 0) && (row < ROW) && (col >= 0) && (col < COL);
}

bool isUnBlocked(int** grid, int row, int col) {
    return (grid[row][col] == 1);
}

bool isDestination(int row, int col, Pair dest) {
    return (row == dest.first && col == dest.second);
}

double calculateHValue(int row, int col, Pair dest) {
    return sqrt((row - dest.first) * (row - dest.first) +
                (col - dest.second) * (col - dest.second));
}

void tracePath(cell** cellDetails, Pair dest) {
    printf("\nThe Path is ");
    int row = dest.first;
    int col = dest.second;

    Pair* Path = (Pair*)malloc(ROW * COL * sizeof(Pair));
    int pathLength = 0;

    while (!(cellDetails[row][col].parent_i == row &&
             cellDetails[row][col].parent_j == col)) {
        Path[pathLength++] = (Pair){row, col};
        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }

    Path[pathLength++] = (Pair){row, col};

    for (int i = pathLength - 1; i >= 0; i--) {
        printf("-> (%d,%d) ", Path[i].first, Path[i].second);
    }
    free(Path);
}

typedef struct {
    pPair* elements;
    int size;
    int capacity;
} OpenList;

void initOpenList(OpenList* list, int capacity) {
    list->elements = (pPair*)malloc(capacity * sizeof(pPair));
    list->size = 0;
    list->capacity = capacity;
}

void insertOpenList(OpenList* list, pPair element) {
    if (list->size == list->capacity) {
        list->capacity *= 2;
        list->elements = (pPair*)realloc(list->elements, list->capacity * sizeof(pPair));
    }
    list->elements[list->size++] = element;
}

pPair extractMin(OpenList* list) {
    int minIndex = 0;
    for (int i = 1; i < list->size; i++) {
        if (list->elements[i].first < list->elements[minIndex].first) {
            minIndex = i;
        }
    }
    pPair minElement = list->elements[minIndex];
    list->elements[minIndex] = list->elements[--list->size];
    return minElement;
}

bool isEmpty(OpenList* list) {
    return list->size == 0;
}

void freeOpenList(OpenList* list) {
    free(list->elements);
}

void aStarSearch(int** grid, Pair src, Pair dest) {
    clock_t start_time = clock();

    if (!isValid(src.first, src.second) || !isValid(dest.first, dest.second)) {
        printf("Source or Destination is invalid\n");
        return;
    }

    if (!isUnBlocked(grid, src.first, src.second) || !isUnBlocked(grid, dest.first, dest.second)) {
        printf("Source or Destination is blocked\n");
        return;
    }

    if (isDestination(src.first, src.second, dest)) {
        printf("Already at the destination\n");
        return;
    }

    bool** closedList = (bool**)malloc(ROW * sizeof(bool*));
    cell** cellDetails = (cell**)malloc(ROW * sizeof(cell*));

    #pragma omp parallel for
    for (int i = 0; i < ROW; i++) {
        closedList[i] = (bool*)calloc(COL, sizeof(bool));
        cellDetails[i] = (cell*)malloc(COL * sizeof(cell));
        for (int j = 0; j < COL; j++) {
            cellDetails[i][j].f = FLT_MAX;
            cellDetails[i][j].g = FLT_MAX;
            cellDetails[i][j].h = FLT_MAX;
            cellDetails[i][j].parent_i = -1;
            cellDetails[i][j].parent_j = -1;
        }
    }

    int i = src.first, j = src.second;
    cellDetails[i][j].f = cellDetails[i][j].g = cellDetails[i][j].h = 0.0;
    cellDetails[i][j].parent_i = i;
    cellDetails[i][j].parent_j = j;

    OpenList openList;
    initOpenList(&openList, 10000);
    insertOpenList(&openList, (pPair){0.0, {i, j}});

    bool foundDest = false;
    int directions[8][2] = {{-1, 0}, {1, 0}, {0, 1}, {0, -1},
                            {-1, 1}, {-1, -1}, {1, 1}, {1, -1}};
    double costs[8] = {1.0, 1.0, 1.0, 1.0, 1.414, 1.414, 1.414, 1.414};

    while (!isEmpty(&openList)) {
        pPair p = extractMin(&openList);
        i = p.second.first;
        j = p.second.second;
        closedList[i][j] = true;

        for (int k = 0; k < 8; k++) {
            int new_i = i + directions[k][0];
            int new_j = j + directions[k][1];

            if (isValid(new_i, new_j)) {
                if (isDestination(new_i, new_j, dest)) {
                    cellDetails[new_i][new_j].parent_i = i;
                    cellDetails[new_i][new_j].parent_j = j;
                    printf("Destination found!\n");
                    tracePath(cellDetails, dest);
                    foundDest = true;

                    clock_t end_time = clock();
                    double time_taken = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
                    printf("\n\nExecution time: %f seconds\n", time_taken);

                    goto cleanup;
                }

                if (!closedList[new_i][new_j] && isUnBlocked(grid, new_i, new_j)) {
                    double gNew = cellDetails[i][j].g + costs[k];
                    double hNew = calculateHValue(new_i, new_j, dest);
                    double fNew = gNew + hNew;

                    if (cellDetails[new_i][new_j].f > fNew) {
                        insertOpenList(&openList, (pPair){fNew, {new_i, new_j}});
                        cellDetails[new_i][new_j].f = fNew;
                        cellDetails[new_i][new_j].g = gNew;
                        cellDetails[new_i][new_j].h = hNew;
                        cellDetails[new_i][new_j].parent_i = i;
                        cellDetails[new_i][new_j].parent_j = j;
                    }
                }
            }
        }
    }

    if (!foundDest) {
        printf("Failed to find the destination\n");
        clock_t end_time = clock();
        printf("Execution time: %f seconds\n",
               ((double)(end_time - start_time)) / CLOCKS_PER_SEC);
    }

cleanup:
    freeOpenList(&openList);
    for (int i = 0; i < ROW; i++) {
        free(closedList[i]);
        free(cellDetails[i]);
    }
    free(closedList);
    free(cellDetails);
}

int main() {
    // Allocate and initialize large grid
    int** grid = (int**)malloc(ROW * sizeof(int*));

    #pragma omp parallel for
    for (int i = 0; i < ROW; i++) {
        grid[i] = (int*)malloc(COL * sizeof(int));
        for (int j = 0; j < COL; j++) {
            grid[i][j] = rand() % 5 == 0 ? 0 : 1; // ~20% blocked
        }
    }

    Pair src = {0, 0};
    Pair dest = {ROW - 1, COL - 1};

    aStarSearch(grid, src, dest);

    for (int i = 0; i < ROW; i++) {
        free(grid[i]);
    }
    free(grid);

    return 0;
}



Writing A_star_openmp.c


In [17]:
!g++ -fopenmp A_star_openmp.c -o A_star_openmp

In [28]:
!./A_star_openmp

Destination found!

The Path is -> (0,0) -> (1,1) -> (2,2) -> (3,3) -> (4,4) -> (5,5) -> (6,6) -> (7,7) -> (8,8) -> (9,9) -> (10,10) -> (11,11) -> (12,12) -> (13,13) -> (14,14) -> (15,15) -> (16,16) -> (17,17) -> (18,18) -> (19,19) -> (20,20) -> (21,21) -> (22,22) -> (23,23) -> (24,24) -> (25,25) -> (26,26) -> (27,27) -> (28,28) -> (28,29) -> (29,30) -> (30,31) -> (31,32) -> (32,33) -> (32,34) -> (33,35) -> (34,36) -> (35,37) -> (36,38) -> (37,39) -> (38,40) -> (39,40) -> (40,41) -> (41,42) -> (42,43) -> (43,44) -> (44,44) -> (45,45) -> (46,46) -> (47,47) -> (48,48) -> (49,49) -> (50,50) -> (51,51) -> (52,52) -> (53,53) -> (54,54) -> (55,55) -> (55,56) -> (56,57) -> (57,58) -> (58,58) -> (59,59) -> (60,60) -> (61,61) -> (62,62) -> (63,63) -> (64,64) -> (65,65) -> (66,65) -> (67,66) -> (68,67) -> (69,68) -> (70,69) -> (71,70) -> (72,71) -> (72,72) -> (73,73) -> (74,74) -> (75,75) -> (76,76) -> (77,77) -> (78,78) -> (79,79) -> (79,80) -> (80,81) -> (81,82) -> (82,83) -> (83,83) -> (84,84

OPENMP OPTIMIZED

In [20]:
%%writefile a_star.c
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <omp.h>
#include <time.h>
#include <float.h>
#include <math.h>

typedef struct {
    int first;
    int second;
} Pair;

typedef struct {
    double f, g, h;
    int parent_i, parent_j;
} Cell;

typedef struct {
    double f;
    Pair coords;
} pPair;

typedef struct {
    pPair* elements;
    int size;
    int capacity;
} PriorityQueue;

PriorityQueue* createPriorityQueue(int capacity) {
    PriorityQueue* pq = (PriorityQueue*)malloc(sizeof(PriorityQueue));
    pq->elements = (pPair*)malloc(capacity * sizeof(pPair));
    pq->size = 0;
    pq->capacity = capacity;
    return pq;
}

void swap(pPair* a, pPair* b) {
    pPair temp = *a;
    *a = *b;
    *b = temp;
}

void heapifyUp(PriorityQueue* pq, int index) {
    while (index > 0 && pq->elements[(index - 1) / 2].f > pq->elements[index].f) {
        swap(&pq->elements[(index - 1) / 2], &pq->elements[index]);
        index = (index - 1) / 2;
    }
}

void heapifyDown(PriorityQueue* pq, int index) {
    int smallest = index;
    int left = 2 * index + 1;
    int right = 2 * index + 2;

    if (left < pq->size && pq->elements[left].f < pq->elements[smallest].f)
        smallest = left;
    if (right < pq->size && pq->elements[right].f < pq->elements[smallest].f)
        smallest = right;

    if (smallest != index) {
        swap(&pq->elements[index], &pq->elements[smallest]);
        heapifyDown(pq, smallest);
    }
}

void insertOpenList(PriorityQueue* pq, pPair element) {
    if (pq->size == pq->capacity) {
        pq->capacity *= 2;
        pq->elements = (pPair*)realloc(pq->elements, pq->capacity * sizeof(pPair));
    }
    pq->elements[pq->size] = element;
    heapifyUp(pq, pq->size);
    pq->size++;
}

pPair extractMin(PriorityQueue* pq) {
    if (pq->size == 0) {
        printf("Priority queue is empty!\n");
        exit(1);
    }
    pPair min = pq->elements[0];
    pq->elements[0] = pq->elements[pq->size - 1];
    pq->size--;
    heapifyDown(pq, 0);
    return min;
}

bool isEmpty(PriorityQueue* pq) {
    return pq->size == 0;
}

#define ROW 1000
#define COL 1000

int directions[8][2] = {
    {-1, 0}, {1, 0}, {0, 1}, {0, -1},
    {-1, 1}, {-1, -1}, {1, 1}, {1, -1}
};

double costs[8] = {1.0, 1.0, 1.0, 1.0, 1.414, 1.414, 1.414, 1.414};

bool isValid(int row, int col) {
    return (row >= 0) && (row < ROW) && (col >= 0) && (col < COL);
}

bool isUnBlocked(int** grid, int row, int col) {
    return grid[row][col] == 1;
}

bool isDestination(int row, int col, Pair dest) {
    return (row == dest.first) && (col == dest.second);
}

double calculateHValue(int row, int col, Pair dest) {
    return sqrt((row - dest.first) * (row - dest.first) +
                (col - dest.second) * (col - dest.second));
}

void tracePath(Cell** cellDetails, Pair dest) {
    // Collect path cells by following parents backward
    int maxPathLength = ROW * COL;
    Pair* path = (Pair*)malloc(maxPathLength * sizeof(Pair));
    int count = 0;

    int row = dest.first;
    int col = dest.second;

    while (!(cellDetails[row][col].parent_i == row && cellDetails[row][col].parent_j == col)) {
        path[count].first = row;
        path[count].second = col;
        count++;

        int temp_row = cellDetails[row][col].parent_i;
        int temp_col = cellDetails[row][col].parent_j;
        row = temp_row;
        col = temp_col;
    }
    // Add the source cell at the end
    path[count].first = row;
    path[count].second = col;
    count++;

    // Print path forward (from source to destination)
    printf("The Path is ");
    for (int i = count - 1; i >= 0; i--) {
        printf("-> (%d,%d) ", path[i].first, path[i].second);
    }
    printf("\n");

    free(path);
}

void aStarSearch(int** grid, Pair src, Pair dest) {
    Cell** cellDetails = (Cell**)malloc(ROW * sizeof(Cell*));
    bool** closedList = (bool**)malloc(ROW * sizeof(bool*));
    for (int i = 0; i < ROW; i++) {
        cellDetails[i] = (Cell*)malloc(COL * sizeof(Cell));
        closedList[i] = (bool*)malloc(COL * sizeof(bool));
        for (int j = 0; j < COL; j++) {
            cellDetails[i][j].f = FLT_MAX;
            cellDetails[i][j].g = FLT_MAX;
            cellDetails[i][j].h = FLT_MAX;
            cellDetails[i][j].parent_i = -1;
            cellDetails[i][j].parent_j = -1;
            closedList[i][j] = false;
        }
    }

    int i = src.first, j = src.second;
    cellDetails[i][j].f = 0.0;
    cellDetails[i][j].g = 0.0;
    cellDetails[i][j].h = 0.0;
    cellDetails[i][j].parent_i = i;
    cellDetails[i][j].parent_j = j;

    PriorityQueue* openList = createPriorityQueue(ROW * COL);
    insertOpenList(openList, (pPair){0.0, {i, j}});

    clock_t start_time = clock();
    volatile bool foundDest = false;

    while (!isEmpty(openList)) {
        pPair p = extractMin(openList);
        i = p.coords.first;
        j = p.coords.second;
        closedList[i][j] = true;

        #pragma omp parallel for shared(foundDest)
        for (int k = 0; k < 8; k++) {
            if (foundDest) continue;

            int new_i = i + directions[k][0];
            int new_j = j + directions[k][1];

            if (isValid(new_i, new_j)) {
                if (isDestination(new_i, new_j, dest)) {
                    #pragma omp critical
                    {
                        if (!foundDest) {
                            foundDest = true;
                            cellDetails[new_i][new_j].parent_i = i;
                            cellDetails[new_i][new_j].parent_j = j;
                            printf("Destination found!\n");
                            tracePath(cellDetails, dest);
                            clock_t end_time = clock();
                            printf("\nExecution time: %f seconds\n",
                                   ((double)(end_time - start_time)) / CLOCKS_PER_SEC);
                        }
                    }
                } else if (!closedList[new_i][new_j] && isUnBlocked(grid, new_i, new_j)) {
                    double gNew = cellDetails[i][j].g + costs[k];
                    double hNew = calculateHValue(new_i, new_j, dest);
                    double fNew = gNew + hNew;

                    #pragma omp critical
                    {
                        if (cellDetails[new_i][new_j].f > fNew) {
                            insertOpenList(openList, (pPair){fNew, {new_i, new_j}});
                            cellDetails[new_i][new_j].f = fNew;
                            cellDetails[new_i][new_j].g = gNew;
                            cellDetails[new_i][new_j].h = hNew;
                            cellDetails[new_i][new_j].parent_i = i;
                            cellDetails[new_i][new_j].parent_j = j;
                        }
                    }
                }
            }
        }

        if (foundDest) break;
    }

    if (!foundDest) {
        printf("Failed to find the Destination Cell\n");
    }

    for (int i = 0; i < ROW; i++) {
        free(cellDetails[i]);
        free(closedList[i]);
    }
    free(cellDetails);
    free(closedList);
    free(openList->elements);
    free(openList);
}

int main() {
    srand(time(NULL));

    int** grid = (int**)malloc(ROW * sizeof(int*));
    for (int i = 0; i < ROW; i++) {
        grid[i] = (int*)malloc(COL * sizeof(int));
        for (int j = 0; j < COL; j++) {
            grid[i][j] = 1; // Initially unblocked
        }
    }

    // Add random obstacles - about 20% blocked cells
    int obstacles = (ROW * COL) / 5; // 20%
    for (int o = 0; o < obstacles; o++) {
        int r = rand() % ROW;
        int c = rand() % COL;
        // Make sure start and destination are not blocked
        if ((r == 0 && c == 0) || (r == ROW - 1 && c == COL - 1))
            continue;
        grid[r][c] = 0;
    }

    Pair src = {0, 0};
    Pair dest = {ROW - 1, COL - 1};

    aStarSearch(grid, src, dest);

    for (int i = 0; i < ROW; i++) {
        free(grid[i]);
    }
    free(grid);

    return 0;
}


Writing a_star.c


In [21]:
!gcc -fopenmp a_star.c -o a_star -lm


In [29]:
!./a_star

Destination found!
The Path is -> (0,0) -> (1,1) -> (2,2) -> (3,3) -> (4,4) -> (5,5) -> (6,6) -> (7,7) -> (8,8) -> (9,9) -> (10,10) -> (11,11) -> (12,12) -> (13,13) -> (14,14) -> (15,15) -> (16,16) -> (17,17) -> (18,18) -> (19,19) -> (20,19) -> (21,19) -> (22,20) -> (23,21) -> (24,22) -> (25,23) -> (26,24) -> (27,25) -> (28,25) -> (29,26) -> (30,27) -> (31,28) -> (32,29) -> (33,30) -> (34,31) -> (35,32) -> (36,33) -> (37,33) -> (38,34) -> (39,35) -> (40,36) -> (41,37) -> (42,38) -> (43,39) -> (44,40) -> (45,41) -> (46,42) -> (47,43) -> (48,44) -> (49,45) -> (50,46) -> (51,46) -> (52,47) -> (53,48) -> (54,49) -> (55,50) -> (56,51) -> (57,52) -> (58,52) -> (59,52) -> (60,53) -> (61,54) -> (62,55) -> (63,56) -> (64,57) -> (65,58) -> (65,59) -> (66,60) -> (67,61) -> (68,62) -> (69,62) -> (70,63) -> (71,63) -> (72,64) -> (73,65) -> (74,66) -> (75,67) -> (76,68) -> (77,69) -> (78,69) -> (79,70) -> (80,71) -> (81,72) -> (82,73) -> (83,74) -> (84,75) -> (85,76) -> (86,77) -> (87,78) -> (88,79)

In [ ]:
!apt-get -qq install -y cuda

Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package liblocale-gettext-perl.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../0-liblocale-gettext-perl_1.07-4build3_amd64.deb ...
Unpacking liblocale-gettext-perl (1.07-4build3) ...
Selecting previously unselected package keyboard-configuration.
Preparing to unpack .../1-keyboard-configuration_1.205ubuntu3_all.deb ...
Unpacking keyboard-configuration (1.205ubuntu3) ...
Selecting previously unselected package cpp-12.
Preparing to unpack .../2-cpp-12_12.3.0-1ubuntu1~22.04_amd64.deb ...
Unpacking cpp-12 (12.3.0-1ubuntu1~22.04) ...
Selecting previously unselected package libasan8:amd64.
Preparing to unpack .../3-libasan8_12.3.0-1ubuntu1~22.04_amd64.deb ...
Unpacking libasan8:amd64 (12.3.0-1ubuntu1~22.04) ...
Selecting previously unselected package libtsan2:amd64.
Preparing to unpack .../4-libtsan2_12.3.0-1ubuntu1~22.04_amd64.deb ...
Unpacki

In [23]:
%%writefile A_star_cuda.cu
#include <iostream>
#include <vector>
#include <queue>
#include <cmath>
#include <cuda_runtime.h>
#include <chrono>
#include <algorithm>


#define N 1000
#define BLOCK_SIZE 256

__global__ void computeHeuristics(float* h_costs) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N * N) {
        int x = idx / N;
        int y = idx % N;
        h_costs[idx] = sqrtf((float)(N - 1 - x) * (N - 1 - x) + (float)(N - 1 - y) * (N - 1 - y));  // Euclidean
    }
}

struct Node {
    int x, y;
    float f, g;
    bool operator>(const Node& other) const {
        return f > other.f;
    }
};

int to1D(int x, int y) {
    return x * N + y;
}

bool isValid(int x, int y) {
    return (x >= 0 && x < N && y >= 0 && y < N);
}

int main() {
    float* h_costs;
    float* d_costs;
    size_t size = N * N * sizeof(float);
    h_costs = (float*)malloc(size);
    cudaMalloc(&d_costs, size);

    int blocks = (N * N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    computeHeuristics<<<blocks, BLOCK_SIZE>>>(d_costs);
    cudaMemcpy(h_costs, d_costs, size, cudaMemcpyDeviceToHost);

    std::priority_queue<Node, std::vector<Node>, std::greater<Node>> open_list;
    std::vector<std::vector<bool>> visited(N, std::vector<bool>(N, false));
    std::vector<std::vector<std::pair<int, int>>> parent(N, std::vector<std::pair<int, int>>(N, {-1, -1}));

    auto start_time = std::chrono::high_resolution_clock::now();

    open_list.push({0, 0, h_costs[to1D(0, 0)], 0});
    int dx[8] = {-1, -1, -1, 0, 1, 1, 1, 0};
    int dy[8] = {-1,  0,  1, 1, 1, 0, -1, -1};

    bool path_found = false;

    while (!open_list.empty()) {
        Node curr = open_list.top();
        open_list.pop();

        if (curr.x == N - 1 && curr.y == N - 1) {
            path_found = true;
            auto end_time = std::chrono::high_resolution_clock::now();
            std::chrono::duration<double> duration = end_time - start_time;

            std::cout << "✅ Path found!" << std::endl;
            std::cout << "🔢 Cost: " << curr.g << std::endl;
            std::cout << "⏱️ Execution Time: " << duration.count() << " seconds\n";
            std::cout << "📍 Shortest Path (from start to goal):" << std::endl;

            // Reconstruct path
            std::vector<std::pair<int, int>> path;
            int x = curr.x;
            int y = curr.y;
            while (x != -1 && y != -1) {
                path.push_back({x, y});
                auto p = parent[x][y];
                x = p.first;
                y = p.second;
            }

            std::reverse(path.begin(), path.end());
            for (const auto& p : path) {
                std::cout << "(" << p.first << ", " << p.second << ") ";
            }
            std::cout << std::endl;

            cudaFree(d_costs);
            free(h_costs);
            return 0;
        }

        if (visited[curr.x][curr.y]) continue;
        visited[curr.x][curr.y] = true;

       for (int i = 0; i < 8; ++i) {
    int nx = curr.x + dx[i];
    int ny = curr.y + dy[i];

    if (isValid(nx, ny) && !visited[nx][ny]) {
        float move_cost = (dx[i] == 0 || dy[i] == 0) ? 1.0f : 1.414f;
        float g_new = curr.g + move_cost;
        float f_new = g_new + h_costs[to1D(nx, ny)];

        open_list.push({nx, ny, f_new, g_new});
        if (parent[nx][ny].first == -1)
            parent[nx][ny] = {curr.x, curr.y};
    }
}

    }

    std::cout << "❌ Path not found." << std::endl;
    cudaFree(d_costs);
    free(h_costs);
    return 0;
}


Overwriting A_star_cuda.cu


In [24]:
!nvcc -arch=sm_75 A_star_cuda.cu -o A_star_cuda

A_star_cuda.cu(59): warning #550-D: variable "path_found" was set but never used
      bool path_found = false;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [30]:
!./A_star_cuda

✅ Path found!
🔢 Cost: 1412.57
⏱️ Execution Time: 0.00395939 seconds
📍 Shortest Path (from start to goal):
(0, 0) (1, 1) (2, 2) (3, 3) (4, 4) (5, 5) (6, 6) (7, 7) (8, 8) (9, 9) (10, 10) (11, 11) (12, 12) (13, 13) (14, 14) (15, 15) (16, 16) (17, 17) (18, 18) (19, 19) (20, 20) (21, 21) (22, 22) (23, 23) (24, 24) (25, 25) (26, 26) (27, 27) (28, 28) (29, 29) (30, 30) (31, 31) (32, 32) (33, 33) (34, 34) (35, 35) (36, 36) (37, 37) (38, 38) (39, 39) (40, 40) (41, 41) (42, 42) (43, 43) (44, 44) (45, 45) (46, 46) (47, 47) (48, 48) (49, 49) (50, 50) (51, 51) (52, 52) (53, 53) (54, 54) (55, 55) (56, 56) (57, 57) (58, 58) (59, 59) (60, 60) (61, 61) (62, 62) (63, 63) (64, 64) (65, 65) (66, 66) (67, 67) (68, 68) (69, 69) (70, 70) (71, 71) (72, 72) (73, 73) (74, 74) (75, 75) (76, 76) (77, 77) (78, 78) (79, 79) (80, 80) (81, 81) (82, 82) (83, 83) (84, 84) (85, 85) (86, 86) (87, 87) (88, 88) (89, 89) (90, 90) (91, 91) (92, 92) (93, 93) (94, 94) (95, 95) (96, 96) (97, 97) (98, 98) (99, 99) (100, 100) (10